In [1]:
import sys
import json
import numpy as np
import pandas as pd

sys.path.append("..")

from graphstructure.simple_substructure import SubstructureSimple
from graphstructure.metrics import MetricFunction, REAL_LIFE_LOADS

In [2]:
REAL_LIFE_LOADS["plausibility"] = ["FAMD GMM", "FAMD Mahalanobis", "one class SVM", "FAMD one class SVM", "kde",
 "isolation forest", "local outlier factor"]  

# REAL_LIFE_LOADS["plausibility"] = ["one class SVM"]          
# NU = 0.5
# metrics = MetricFunction.from_loads(REAL_LIFE_LOADS, nu=NU)

REAL_LIFE_LOADS["plausibility"] = None          

metrics = MetricFunction.from_loads(REAL_LIFE_LOADS)

Running with objectives: ['Cost', 'wave', 'combined', 'compression', 'tipover', 'torque']


In [3]:
import math
def test_all(designs):
    objs = [] 
    evaluated_designs = []
    for d in designs:
        sub = SubstructureSimple.from_dict(d)
        obj_values = np.array(metrics(sub)[0])
        if any([math.isinf(x) for x in obj_values]):
            print('This design cant be tested skipping:', d)
            continue
            # print('This design cant be tested in ops and thus got scores of 0.0:', d)
            # obj_values[np.where(obj_values == 0.0)] = np.inf
        objs.append(obj_values)
        evaluated_designs.append(d)
    return objs, evaluated_designs

In [4]:
def to_df(objs):
    # print(metrics.metric_names)
    return pd.DataFrame(objs, columns=metrics.metric_names)

In [5]:
import os
def evaluate(designs, file_path, out_dir):
    objs, evaluated_designs = test_all(designs)
    df = to_df(objs)
    data_merged = [{**d, **df.loc[i]} for i, d in enumerate(evaluated_designs)]
    filename = os.path.splitext(os.path.basename(file_path))[0]
    output_filename = out_dir + filename +"_evaluated.json"
    with open(output_filename, "w") as json_file:
        json.dump(data_merged, json_file, indent=4)
    print("Saved file", output_filename, "total designs", len(evaluated_designs))    

In [6]:
out_dir = "data/evaluated/"
in_dir = "data/to_evaluate/"
files = os.listdir(in_dir)
out_files = os.listdir(out_dir)
print(out_files)
for file_ in files:
    if file_.split(".json")[0] + "_evaluated.json" not in out_files:
        with open(in_dir + file_) as f:
            synthetic = json.load(f)
            evaluate(synthetic, file_, out_dir)


[]
Saved file data/evaluated/real_structures_evaluated.json total designs 100
